# **$K_p$**

# Installing third-party packages

In [ ]:
%%capture
!pip install --prefer-binary pyscf
!pip install pyscf[geomopt]

In [14]:
from pyscf import gto, scf
from pyscf.geomopt.geometric_solver import optimize

mol = gto.M(atom='N 0 0 0; N 0 0 1.2', basis='3-21g')
mf = mol.RKS(xc='pbe')

# geometric
mol_eq = optimize(mf, maxsteps=100)
print(mol_eq.atom_coords())

geometric-optimize called with the following command line:
/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-0a736671-b40b-4741-bd27-64a285b548a1.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   N   0.000000   0.000000   1.200000    0.000000  0.000000  0.000000
converged SCF energy = -108.764652693871
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 N     0.0000000000    -0.0000000000    -0.1287833262
1 N     0.0000000000     0.0000000000     0.1287833262
----------------------------------------------
cycle 1: E = -108.764652694  dE = -108.765  norm(grad) = 0.182127


Step    0 : Gradient = 1.288e-01/1.288e-01 (rms/max) Energy = -108.7646526939
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 5.00000e-02 5.00000e-02 1.04823e+00



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N  -0.000000   0.000000   0.032507   -0.000000  0.000000  0.032507
   N  -0.000000  -0.000000   1.167493   -0.000000 -0.000000 -0.032507
converged SCF energy = -108.77426966539
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 N     0.0000000000     0.0000000000    -0.0176220518
1 N     0.0000000000    -0.0000000000     0.0176220518
----------------------------------------------
cycle 2: E = -108.774269665  dE = -0.00961697  norm(grad) = 0.0249213


Step    1 : Displace = 3.251e-02/3.251e-02 (rms/max) Trust = 1.000e-01 (=) Grad = 1.762e-02/1.762e-02 (rms/max) E (change) = -108.7742696654 (-9.617e-03) Quality = 1.216
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 5.00000e-02 5.00000e-02 9.04796e-01


...


Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N  -0.000000   0.000000   0.036326   -0.000000  0.000000 -0.001334
   N  -0.000000  -0.000000   1.163674    0.000000 -0.000000  0.001334
converged SCF energy = -108.77439962554
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 N     0.0000000000    -0.0000000000    -0.0001881821
1 N     0.0000000000     0.0000000000     0.0001881821
----------------------------------------------
cycle 4: E = -108.774399626  dE = -1.49325e-05  norm(grad) = 0.00026613


Step    3 : Displace = 1.334e-03/1.334e-03 (rms/max) Trust = 1.414e-01 (=) Grad = 1.882e-04/1.882e-04 (rms/max) E (change) = -108.7743996255 (-1.493e-05) Quality = 0.962
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 5.00000e-02 5.00000e-02 1.25815e+00



Geometry optimization cycle 5
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N  -0.000000   0.000000   0.036366   -0.000000  0.000000  0.000040
   N  -0.000000  -0.000000   1.163634   -0.000000 -0.000000 -0.000040
converged SCF energy = -108.774399641307
--------------- RKS_Scanner gradients ---------------
         x                y                z
0 N     0.0000000000    -0.0000000000    -0.0000025527
1 N    -0.0000000000     0.0000000000     0.0000025527
----------------------------------------------
cycle 5: E = -108.774399641  dE = -1.57671e-08  norm(grad) = 3.60999e-06


Step    4 : Displace = 3.957e-05/3.957e-05 (rms/max) Trust = 2.000e-01 (+) Grad = 2.553e-06/2.553e-06 (rms/max) E (change) = -108.7743996413 (-1.577e-08) Quality = 1.120
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 5.00000e-02 5.00000e-02 1.25815e+00
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 5.578 seconds


[[-3.61226049e-14  5.87811900e-14  6.87209527e-02]
 [-2.90128559e-14 -4.20674136e-14  2.19895040e+00]]
